# Imports and Functions

In [1]:
import glob
from PIL import ImageDraw
from PIL import Image
import numpy as np
import re
import codecs
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import ntpath

## Hyperparameters

In [2]:
OUTSIZE = (105, 75)

SCALE_WIDTH_CM = 2. # width in cm between the two outer scale points
SCALE_PADDING_FACTOR = 1.4 # factor to widen the width of the box 
                           # in relation to the distance between the scale points.
    
                           # Without padding, the scale points show a distance of 2cm.
                           # However, the scales are drawn at least 0.4cm longer on each side.
                           # Due to padding, the box has a width of 2.8cm = 1.4*2cm.

NUM_SEGMENTS = 3     # An odd-numbered number of segments that determines which parts are being cut.
                     # The segments overlie one another to 50%.   

# Extra Augmentationen:
NUM_TRANSLATIONS = 9 # Number of random tranlations that are applied to each segment
TRANSLATION_DELTA = 30   # Translations are randomly selected out of the interval
                         # [-TRANSLATION_DELTA, TRANSLATION_DELTA]
                       
SCALE_PATH = 'all_scale_data.npy'
INFOLDER = 'DIP_images_fresh/all/'
OUTFOLDER = f"images_classification/"

## Function to calculate in an affine plane

In [3]:
#Translation of radian measure in degree
def rad_to_deg(rad):
    return rad * 360 / (2*np.pi) 

#Translation of degree in radian measure
def deg_to_rad(deg):
    return deg / 360 * (2*np.pi)

Apply the affine rotation
$\mathbb{A}_{\varphi}(v) = O_{\varphi}v + m$
with rotationmatrix $O_{\varphi} = \begin{pmatrix} \operatorname{cos}(\varphi) & \operatorname{sin}(\varphi) \\ - \operatorname{sin}(\varphi) & \operatorname{cos}(\varphi) \end{pmatrix}$ around the middle point $m$ on several vectors $v = \text{points}$.

In [ ]:
# Updates coordinates of points after the rotation around the given middle by the given angle.
def affine_rotation(angle, middle, points):
    
    phi = deg_to_rad(angle) #degree to radian measure
    
    O = np.array([[np.cos(phi), np.sin(phi)],[-np.sin(phi), np.cos(phi)]]).reshape([2,2]) #rotation matrix
    v = points - middle
    
    flat = len(v.shape) == 1
    
    if flat:
        v = v.reshape([1,2])
        
    #get new cooedinated by multiplication with rotation matrix and addition of the middle
    result = np.einsum('ij,kj -> ki', O, v) + middle      
    
    if flat:
        result = result.flatten()
    
    return result


Apply the inverse affine rotation
$\mathbb{A}_{\varphi}(v) = O^\intercal_{\varphi}v + m$
with rotationmatrix $O^\intercal_{\varphi} = \begin{pmatrix} \operatorname{cos}(\varphi) & -\operatorname{sin}(\varphi) \\ \operatorname{sin}(\varphi) & \operatorname{cos}(\varphi) \end{pmatrix}$ around the middle point $m$ on several vectors $v = \text{points}$.

In [5]:
def inverse_affine_rotation(angle, middle, points):
    
    phi = deg_to_rad(angle) #degree in radian measure
    
    O = np.array([[np.cos(phi), np.sin(phi)],[-np.sin(phi), np.cos(phi)]]).reshape([2,2]) #rotation matrix
    v = points - middle
    
    flat = len(v.shape) == 1
    
    if flat:
        v = v.reshape([1,2])
        
    #get new cooedinated by multiplication with rotation matrix and addition of the middle
    result = np.einsum('ij,kj -> ki', O.T, v) + middle     
    
    if flat:
        result = result.flatten()
    
    return result

Calculate the centroid s = $\frac{1}{m}\cdot\sum\limits_{i=1}^{m} x_{i} $

In [6]:
def centroid(points):
    return np.mean(points, axis=0)

Translate the origin point to new_origin and update given points

In [7]:
def affine_translation(new_origin, points):
    return points - new_origin

# Processing functions

## Calculate rotanial affinity for given scale points of an image

In [8]:
def affine_rotation_from_scale_data(img_dimension, scale_data, top_offset = 300):
    
    left = scale_data['left']
    right = scale_data['right']
    x1 = left[0]
    y1 = left[1]
    x2 = right[0]
    y2 = right[1]
    
    l = np.array([x1,y1])
    r = np.array([x2,y2])
    angle = 0

    if (y2 != y1):

        alpha = np.arctan(np.abs(y2-y1) / np.abs(x2-x1)) #calculate the angle
        angle = rad_to_deg(alpha) #radian measure to degree
        angle = -np.sign(y2-y1) * angle #get sign to rotate accordingly

    middle = np.array( [img_dimension[0] / 2, img_dimension[1] / 2] ) #calculate the middle of the image, 
                                                                      # because rotate function rotates 
                                                                      # around the middle point
    return angle, middle

## Use the above rotation to determine a box in order to cut off the clothes-pegs

In [9]:
def crop_from_scale_affinity(img, angle, middle, scale_data, top_offset = 300, scale_padding_factor=SCALE_PADDING_FACTOR):
    
    l = np.array(scale_data['left'])
    r = np.array(scale_data['right'])
    
    l[1] = img.height-l[1] #get points in the scale with (0,0) in the upper left corner
    r[1] = img.height-r[1] 
    
    scale_data = np.array([ [l[0], l[1]], [r[0], r[1]] ])
    scale_data_rotated = affine_rotation(angle, middle, scale_data) #get the rotated coordinates
    lnew = scale_data_rotated[0,:]
    rnew = scale_data_rotated[1,:]
    
    img = img.rotate(angle) #rotate the image 

    v = rnew - lnew #calculate distance between both scale points
    lnew = lnew - (scale_padding_factor - 1)/2*v # widen the width
    rnew = rnew + (scale_padding_factor - 1)/2*v 
    w = np.array([ v[1], -v[0] ]) #canonical choice for a vector perpendicular to v
    ulnew = lnew + w #add w to the furthest left point
    ulnew[1] = ulnew[1] - top_offset #increase the height such that no part of the shoot gets cut off

    box = (ulnew[0],ulnew[1],rnew[0],rnew[1]) #define box

    cropped = img.crop(box) #crop image

    return cropped, box

## Determine an affine rotation based on a regression over the green pixel points of a shoot

In [10]:
def affine_rotation_from_scale_data_crop(cropped):
    im_rgb = cropped.convert("RGB")
    I = np.transpose(np.array(im_rgb), (1,0,2))
    I = I.astype(np.int64)

    # treshold for the colour channel in order to detect the white background
    wlim = 10

    # Determine the colour channel
    R = I[:,:,0]
    G = I[:,:,1]
    B = I[:,:,2]

    # Set relations of the channel to detect the white background
    W = np.max( np.c_[ np.abs(R-G)[:,:,np.newaxis], np.abs(B-G)[:,:,np.newaxis], np.abs(R-B)[:,:,np.newaxis] ], axis=2)

    # Calculate the framing mask to detect the pixels that belong to a shoot
    J = (R <= G) & (B <= G) & (W > wlim) 
    
    x,y = J.nonzero()
    reg = LinearRegression(fit_intercept = True).fit(x[:,np.newaxis],y) #calculate a linear regression over all these points
    
    reg_coef = reg.coef_[0]
    reg_data = (reg.coef_, reg.intercept_)
    
    angle = np.arctan(1 / np.abs(reg_coef)) * 180 / np.pi #calculate an angle basen on the regression coefficient
    angle = np.sign(reg_coef) * (90-angle) #correct the sign of the angle
    middle = np.array( [cropped.width / 2, cropped.height / 2] ) #get the middel point

    return angle, middle, reg_data

### Für Testzwecke

In [11]:
def draw_landmarks(img, landmarks, radius = 10):
    lm = landmarks
    draw = ImageDraw.Draw(img)
    for i in range(lm.shape[0]):
        draw.ellipse((lm[i,0]-radius,lm[i,1]-radius, lm[i,0]+radius,lm[i,1]+radius),fill = 'red')
    
    return img

In [12]:
scale_data = np.load(SCALE_PATH, allow_pickle = True)
scale_data = scale_data[()]
inpaths = glob.glob(INFOLDER + '*.jpg')

outratio = OUTSIZE[0]/OUTSIZE[1]       
n = len(inpaths)   
# Randomly chosen translations for x- and y-coordinates
translations = np.random.randint(low=-TRANSLATION_DELTA,high=TRANSLATION_DELTA, size = (n,NUM_TRANSLATIONS,2))
transformed_landmarks = {}

DEBUG_STEPS = 5
debug_interval = int(DEBUG_STEPS/100 * 666)

for i in range(n):
    if i % debug_interval == 0:
        print(f"{i // debug_interval * DEBUG_STEPS}%")
            
    inpath = inpaths[i]
    filename = inpath[len(INFOLDER):]
    img = Image.open(inpath)
   
    a1, m1 = affine_rotation_from_scale_data(img.size, scale_data[filename])
    cropped, box = crop_from_scale_affinity(img, a1, m1, scale_data[filename])
    a2, m2_local, reg_data = affine_rotation_from_scale_data_crop(cropped)
    
    # Calculate the left_middle point of the shoot for segmentation
    #Therefore, get the scale points and replicate steps from above
    l = np.array(scale_data[filename]['left'])
    r = np.array(scale_data[filename]['right'])
    l[1] = img.height-l[1]
    r[1] = img.height-r[1] 
    
    sd = np.array([ [l[0], l[1]], [r[0], r[1]] ])
    sd_rotated = affine_rotation(a1, m1, sd)
    lnew = sd_rotated[0,:]
    rnew = sd_rotated[1,:]
    # Calculate the distance 
    v = rnew - lnew
    # Widen the width
    lnew = lnew - (SCALE_PADDING_FACTOR - 1)/2*v
    
    # local coordinates in the box
    lnew = lnew - np.array(box[:2])
    
    # get local coordinate of the middle of the shoot with the regression data
    lnew[1] = reg_data[0][0]*lnew[0] + reg_data[1]
    
    # Transform to global rotated coordinate system
    lnew = lnew + np.array(box[:2])
    
    # Transform to coordinates in the global original system
    lnew = np.array(lnew).reshape((1,2))
    left_middle = inverse_affine_rotation(a1,m1,lnew)
    
    #Add both angles
    angle = a1 + a2
    #Calculate relation of pixels to cm 
    box_width_in_cm = SCALE_PADDING_FACTOR * SCALE_WIDTH_CM
    cm_pixel_ratio = (box[2] - box[0]) / box_width_in_cm #x/2.8 = 1cm

    #get middle of the image
    middle_image = np.array(img.size) / 2
    
    #rotate the image
    img = img.rotate(angle)
    
    #get the wanted left_middle point
    left_middle = affine_rotation(angle, middle_image, left_middle).flatten()
    
    # determine the width and height of a segment in cm 
    cm_width = SCALE_PADDING_FACTOR * SCALE_WIDTH_CM / ( (NUM_SEGMENTS-1) / 2 + 1 )
    cm_height = cm_width / outratio 
    
    #calculate the box for each segment
    for s in range(NUM_SEGMENTS):
    
        w_left = np.array([s*cm_pixel_ratio * cm_width / 2, -(cm_pixel_ratio*cm_height/2)])
        w_right = np.array([w_left[0] + cm_pixel_ratio * cm_width , -w_left[1]])

        left = left_middle + w_left 
        right = left_middle + w_right

        #get translations
        T = np.array(np.r_[np.matrix(((0,0))), translations[i,...]])
        
        #apply all translations on each segment
        for t in range(NUM_TRANSLATIONS+1):
            translation = T[t,:]

            left_t = left + translation
            right_t = right + translation

            box = (left_t[0],left_t[1],right_t[0],right_t[1]) #determine the final box

            cropped = img.crop(box) #crop the image

            cropped = cropped.resize(OUTSIZE) #resize the image

            filename_t = filename[:-len(".jpg")] + f"_s{s}_t{t}.jpg"
            cropped.save(OUTFOLDER + filename_t)

0%
5%
10%
15%
20%
25%
30%
35%
40%
45%
50%
55%
60%
65%
70%
75%
80%
85%
90%
95%
100%
